# Poisson regression example with offsets

Based on [Stan counting example](https://mc-stan.org/rstanarm/articles/count.html) which in turn comes from an example form Gelman and Hill (repeated also in Gelman Hill and Vehtari).

TODO: fix these references.  Perhaps focus on ROS example .

In [7]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from scipy.stats import nbinom

In [17]:
roaches = pd.read_csv("data/roaches.csv")
# rescale 
roaches["roach1"] = roaches["roach1"] / 100
roaches.head()


,Unnamed: 0,y,roach1,treatment,senior,exposure2
0,1,153,3.0800,1,0,0.800000
1,2,127,3.3125,1,0,0.600000
2,3,7,0.0167,1,0,1.000000
3,4,7,0.0300,1,0,1.000000
4,5,0,0.0200,1,0,1.142857


In [18]:


# bambi poisson model
model = bmb.Model("y ~ roach1 + treatment  + senior + offset(log(exposure2))", family = "poisson", data = roaches)
results = model.fit()


Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [Intercept, roach1, treatment, senior]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 1 seconds.


In [19]:
az.summary(results)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,3.089,0.021,3.049,3.128,0.000,0.0,3879.0,3330.0,1.0
roach1,0.698,0.009,0.683,0.715,0.000,0.0,3216.0,3110.0,1.0
senior,-0.381,0.034,-0.445,-0.317,0.001,0.0,4027.0,3155.0,1.0
treatment,-0.518,0.025,-0.565,-0.472,0.000,0.0,3886.0,3423.0,1.0


Very good agreement with the stan model.  

In ROS they use neg binomial. 15.2 

In [20]:
# bambi poisson model
model = bmb.Model("y ~ roach1 + treatment  + senior + offset(log(exposure2))", family = "negativebinomial", data = roaches)
results = model.fit()

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, Intercept, roach1, treatment, senior]


Output()

Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 1 seconds.


In [21]:
az.summary(results)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
Intercept,2.849,0.233,2.406,3.271,0.003,0.002,5386.0,3438.0,1.0
alpha,0.272,0.026,0.223,0.320,0.000,0.000,5241.0,3625.0,1.0
roach1,1.315,0.258,0.843,1.817,0.004,0.003,5178.0,3274.0,1.0
senior,-0.333,0.272,-0.846,0.173,0.004,0.003,5673.0,3003.0,1.0
treatment,-0.775,0.252,-1.236,-0.289,0.004,0.003,4826.0,3241.0,1.0


Good agreement with ROS results